# Model-Based Collaborative Filtering: Matrix Factorization 

## Importing Necessary Libraries

In [2]:
import numpy as np
import pandas as pd
from surprise import Reader, SVD, Dataset, accuracy
from surprise.model_selection import GridSearchCV, train_test_split, cross_validate
pd.set_option('display.max_columns', 30)
pd.set_option('display.width', 100)


import warnings
warnings.filterwarnings('ignore')


## Reading datasets

In [3]:
df= pd.read_csv('df_final_21_09___9794.csv',index_col=0)
df

,book_id,book_title,book_series,title,book_authors,genres,book_score,book_rating,book_rating_obj,book_rating_count,book_review_count,book_desc,tags,FE_text,book_desc_tags_FE,...,book_edition,book_format,original_publication_year,language_code,book_pages,book_pages_obj,books_count,books_count_obj,goodreads_book_id,book_isbn,isbn,isbn13,image_url_x,image_url_y,small_image_url
0,1,The Hunger Games,"The Hunger Games, #1","The Hunger Games (The Hunger Games, #1)",Suzanne Collins,Young Adult|Fiction|Science Fiction|Dystopia|F...,1.000000e+00,4.33,4.0,5519135,160706,Winning will make you famous. Losing means cer...,to-read fantasy favorites currently-reading yo...,The-Hunger-Games Suzanne-Collins Young-Adult F...,Winning will make you famous. Losing means cer...,...,NaN,Hardcover,2008.0,eng,374.0,8.0,272,14.0,2767052,9.780440e+12,439023483,9.780439e+12,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603l...,https://images.gr-assets.com/books/1447303603s...
1,2,Harry Potter and the Philosopher's Stone,"Harry Potter, #1",Harry Potter and the Sorcerer's Stone (Harry P...,"J.K. Rowling, Mary GrandPré",NaN,8.245693e-01,4.44,4.0,6682936,108361,NaN,to-read fantasy favorites currently-reading yo...,Harry-Potter-and-the-Philosopher's-Stone J.K.-...,to-read fantasy favorites currently-reading y...,...,NaN,NaN,1997.0,eng,NaN,NaN,491,15.0,3,NaN,439554934,9.780440e+12,https://images.gr-assets.com/books/1474154022m...,NaN,https://images.gr-assets.com/books/1474154022s...
2,3,Twilight,"Twilight, #1","Twilight (Twilight, #1)",Stephenie Meyer,Young Adult|Fantasy|Romance|Paranormal|Vampire...,2.902011e-02,3.58,3.5,4281268,97991,About three things I was absolutely positive.F...,to-read fantasy favorites currently-reading yo...,Twilight Stephenie-Meyer Young-Adult Fantasy R...,About three things I was absolutely positive.F...,...,NaN,Paperback,2005.0,en-US,498.0,9.0,226,14.0,41865,9.780320e+12,316015849,9.780316e+12,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443l...,https://images.gr-assets.com/books/1361039443s...
3,4,To Kill a Mockingbird,-,To Kill a Mockingbird,Harper Lee,Classics|Fiction|Historical|Historical Fiction...,2.310594e-01,4.27,4.0,3745197,79450,The unforgettable novel of a childhood in a sl...,to-read favorites currently-reading young-adul...,To-Kill-a-Mockingbird Harper-Lee Classics Fict...,The unforgettable novel of a childhood in a sl...,...,50th Anniversary,Paperback,1960.0,eng,324.0,8.0,487,15.0,2657,9.780060e+12,61120081,9.780061e+12,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680l...,https://images.gr-assets.com/books/1361975680s...
4,5,The Great Gatsby,-,The Great Gatsby,F. Scott Fitzgerald,Classics|Fiction|Academic|School|Literature|Hi...,4.942671e-02,3.90,3.5,3141842,56953,Alternate Cover Edition ISBN: 0743273567 (ISBN...,to-read favorites currently-reading young-adul...,The-Great-Gatsby F.-Scott-Fitzgerald Classics ...,Alternate Cover Edition ISBN: 0743273567 (ISBN...,...,US / CAN,Paperback,1925.0,eng,180.0,5.0,1356,15.0,4671,NaN,743273567,9.780743e+12,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560l...,https://images.gr-assets.com/books/1490528560s...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9789,9996,Bayou Moon,"The Edge, #2","Bayou Moon (The Edge, #2)",Ilona Andrews,Fantasy|Urban Fantasy|Fantasy|Romance|Fantasy|...,5.206229e-06,4.10,4.0,21145,1310,"The Edge lies between worlds, on the border be...",to-read fantasy favorites currently-reading fi...,Bayou-Moon Ilona-Andrews Fantasy Urban-Fantasy...,"The Edge lies between worlds, on the border be...",...,NaN,Mass Market Paperback,2010.0,eng,447.0,9.0,19,4.0,7130616,9.780440e+12,441019455,9.780441e+12,https://images.gr-assets.com/books/1307445460m...,https://images.gr-assets.com/books/1307445460l...,https://

In [4]:
df_ratings = pd.read_csv('ratings.csv',index_col=0)
df_ratings=df_ratings.reset_index()
df_ratings

,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3
...,...,...,...
5976474,49925,510,5
5976475,49925,528,4
5976476,49925,722,4
5976477,49925,949,5


## Data Preparation

In [5]:
df_ratings=df_ratings.merge(df[["book_id","book_title"]],on="book_id")  
df_ratings

,user_id,book_id,rating,book_title
0,1,258,5,La sombra del viento
1,11,258,3,La sombra del viento
2,143,258,4,La sombra del viento
3,242,258,5,La sombra del viento
4,325,258,4,La sombra del viento
...,...,...,...,...
5911084,36773,9548,5,Vicious
5911085,50673,9548,4,Vicious
5911086,45213,9548,3,Vicious
5911087,12872,9548,4,Vicious


In [6]:
df_ratings[df_ratings["user_id"]==52036].head(20)

,user_id,book_id,rating,book_title
46067,52036,70,4,Ender's Game
50271,52036,264,5,The Sun Also Rises
68371,52036,18,2,Harry Potter and the Prisoner of Azkaban
83454,52036,27,2,Harry Potter and the Half-Blood Prince
98742,52036,21,2,Harry Potter and the Order of the Phoenix
120602,52036,2,3,Harry Potter and the Philosopher's Stone
136200,52036,23,2,Harry Potter and the Chamber of Secrets
153305,52036,964,5,The Hobbit and The Lord of the Rings
158767,52036,103,5,Le Comte de Monte-Cristo
171853,52036,35,3,O Alquimista


In [7]:
user_book_df = df_ratings.pivot_table(index=["user_id"],
                                      columns=["book_id"],
                                      values="rating")

user_book_df.shape

(53424, 9794)

In [8]:
user_movie_df

book_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,...,9986,9987,9988,9989,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999,10000
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,4.0,5.0,NaN,4.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,5.0,NaN,NaN,5.0,NaN,NaN,4.0,NaN,5.0,NaN,NaN,NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,5.0,NaN,4.0,4.0,NaN,4.0,4.0,NaN,5.0,4.0,NaN,4.0,3.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53420,4.0,5.0,3.0,NaN,2.0,NaN,NaN,NaN,4.0,3.0,3.0,NaN,NaN,3.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53421,4.0,5.0,NaN,5.0,4.0,NaN,4.0,NaN,5.0,NaN,5.0,5.0,NaN,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
53422,4.0,5.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,5.0,NaN,4.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 2. Modelleme

In [9]:
reader = Reader(rating_scale=(1, 5)) #reder skalamız 1 ile 5 arasında diyoruz
data = Dataset.load_from_df(df_ratings[['user_id','book_id','rating']], reader)

In [10]:
trainset, testset = train_test_split(data, test_size=.25)
svd_model = SVD()
svd_model.fit(trainset)
predictions = svd_model.test(testset)
predictions[:3]

[Prediction(uid=14092, iid=276, r_ui=2.0, est=4.166086393358283, details={'was_impossible': False}),
 Prediction(uid=5899, iid=2367, r_ui=4.0, est=3.799314722256897, details={'was_impossible': False}),
 Prediction(uid=25991, iid=520, r_ui=5.0, est=4.714509159589415, details={'was_impossible': False})]

In [11]:
accuracy.rmse(predictions)

RMSE: 0.8335


0.8335204867594611

In [12]:
df_pred=pd.DataFrame(predictions)
df_pred

,uid,iid,r_ui,est,details
0,14092,276,2.0,4.166086,{'was_impossible': False}
1,5899,2367,4.0,3.799315,{'was_impossible': False}
2,25991,520,5.0,4.714509,{'was_impossible': False}
3,36104,4267,5.0,4.407958,{'was_impossible': False}
4,35889,142,3.0,3.596076,{'was_impossible': False}
...,...,...,...,...,...
1477768,17848,153,5.0,4.282944,{'was_impossible': False}
1477769,12813,1049,3.0,3.390432,{'was_impossible': False}
1477770,11131,69,5.0,4.893787,{'was_impossible': False}
1477771,31961,4695,4.0,3.952444,{'was_impossible': False}


In [13]:
df_pred["hata"]=abs(df_pred["r_ui"]-df_pred["est"])
df_pred

,uid,iid,r_ui,est,details,hata
0,14092,276,2.0,4.166086,{'was_impossible': False},2.166086
1,5899,2367,4.0,3.799315,{'was_impossible': False},0.200685
2,25991,520,5.0,4.714509,{'was_impossible': False},0.285491
3,36104,4267,5.0,4.407958,{'was_impossible': False},0.592042
4,35889,142,3.0,3.596076,{'was_impossible': False},0.596076
...,...,...,...,...,...,...
1477768,17848,153,5.0,4.282944,{'was_impossible': False},0.717056
1477769,12813,1049,3.0,3.390432,{'was_impossible': False},0.390432
1477770,11131,69,5.0,4.893787,{'was_impossible': False},0.106213
1477771,31961,4695,4.0,3.952444,{'was_impossible': False},0.047556


In [14]:
df_pred["hata"].mean()

0.6437980896471797

In [15]:
df_pred["hata"].median()

0.528121287092584

In [16]:
svd_model.predict(uid=52036, iid=4, verbose=True)

user: 52036      item: 4          r_ui = None   est = 4.19   {'was_impossible': False}


Prediction(uid=52036, iid=4, r_ui=None, est=4.1945073582973, details={'was_impossible': False})

In [17]:
df_ratings[df_ratings["user_id"] == 52036]

,user_id,book_id,rating,book_title
46067,52036,70,4,Ender's Game
50271,52036,264,5,The Sun Also Rises
68371,52036,18,2,Harry Potter and the Prisoner of Azkaban
83454,52036,27,2,Harry Potter and the Half-Blood Prince
98742,52036,21,2,Harry Potter and the Order of the Phoenix
...,...,...,...,...
5134112,52036,1027,2,The Blood of Olympus
5147593,52036,252,1,Cinder
5307683,52036,218,1,Bared to You
5441517,52036,452,2,The 5th Wave


# Adım 3: Model Tuning

In [18]:
# Çok uzun sürdüğü için şimdilik yapmadım

# ##############################
# # Adım 3: Model Tuning
# ##############################

# param_grid = {'n_epochs': [5, 10, 20],
#               'lr_all': [0.002, 0.005, 0.007]}


# gs = GridSearchCV(SVD,
#                   param_grid,
#                   measures=['rmse', 'mae'],
#                   cv=3,
#                   n_jobs=-1,
#                   joblib_verbose=True)

# gs.fit(data)

# gs.best_score['rmse']

In [19]:
# gs.best_params['rmse']

In [20]:
##############################
# Adım 4: Final Model ve Tahmin
##############################

In [21]:
# # iki yıldız arkasına sözlük nesnesini gönderirsek bu sözlük nesnesindeki parametreler girilmiş olur
# svd_model = SVD(**gs.best_params['rmse']) 

# data = data.build_full_trainset() # en son  tüm model ile eğittik
# svd_model.fit(data)

In [22]:
# svd_model.predict(uid=1.0, iid=541, verbose=True)

# Adım 4: Final Model ve Tahmin

In [23]:
data2 = data.build_full_trainset() # en son  tüm model ile eğittik
svd_model.fit(data2)

In [24]:
df_best=df[df["book_rating"]>df["book_rating"].median()][df["book_score"]>df["book_score"].median()][["book_id","book_title","book_authors"]]

In [25]:
df_bookish_user=df_ratings["user_id"].value_counts()
df_bookish_user=df_bookish_user[df_bookish_user>150].index
df_bookish_user

Int64Index([12381, 28158, 30944, 12874, 52036, 45554,  9668, 14372, 24143, 38798,
            ...
            18621, 20271, 12530, 13479, 39181, 11362,  5676, 17408, 35477, 38206],
           dtype='int64', length=3710)

In [26]:
df_bookish_user[1:5]

Int64Index([28158, 30944, 12874, 52036], dtype='int64')

In [27]:
df_best.iloc[:10]

,book_id,book_title,book_authors
0,1,The Hunger Games,Suzanne Collins
1,2,Harry Potter and the Philosopher's Stone,"J.K. Rowling, Mary GrandPré"
3,4,To Kill a Mockingbird,Harper Lee
5,6,The Fault in Our Stars,John Green
6,7,The Hobbit or There and Back Again,J.R.R. Tolkien
9,10,Pride and Prejudice,Jane Austen
10,11,The Kite Runner,Khaled Hosseini
11,12,Divergent,Veronica Roth
12,13,Nineteen Eighty-Four,"George Orwell, Erich Fromm, Celâl Üster"
14,15,Het Achterhuis: Dagboekbrieven 14 juni 1942 - ...,"Anne Frank, Eleanor Roosevelt, B.M. Mooyaart-D..."


In [28]:
user_id=52036

preds2=[]
for book_id in df_best["book_id"].unique() :
    est=svd_model.predict(uid=user_id, iid=book_id, verbose=False).est
    #print(user_id,book_id,est)
    preds2.append([user_id,book_id,est])

In [29]:
preds=pd.DataFrame(preds2,columns=["user_id","book_id","est"])
preds=preds.sort_values(by="est",ascending=False)
preds=preds.merge(df[["book_id","title"]],on=["book_id"])
preds.iloc[:15]

,user_id,book_id,est,title
0,52036,109,5.000000,Les Misérables
1,52036,177,4.980231,Crime and Punishment
2,52036,103,4.805161,The Count of Monte Cristo
3,52036,13,4.722912,1984
4,52036,498,4.686479,War and Peace
5,52036,1367,4.681335,The Once and Future King (The Once and Future ...
6,52036,555,4.667246,"Hamlet: Screenplay, Introduction And Film Diary"
7,52036,2142,4.657334,The Iliad/The Odyssey
8,52036,225,4.618217,East of Eden
9,52036,964,4.615407,J.R.R. Tolkien 4-Book Boxed Set: The Hobbit an...


In [30]:
df_ratings[df_ratings["user_id"]==user_id].sort_values("rating",ascending=False)

,user_id,book_id,rating,book_title
1813446,52036,276,5,Foundation
1890130,52036,116,5,The Adventures of Tom Sawyer
1795843,52036,1014,5,"So Long, and Thanks for All the Fish"
998162,52036,7,5,The Hobbit or There and Back Again
900787,52036,977,5,Inferno
...,...,...,...,...
4559968,52036,202,1,"Hush, Hush"
1022818,52036,75,1,Bridget Jones's Diary
5147593,52036,252,1,Cinder
5307683,52036,218,1,Bared to You


# est değerlerinin tahmini 

In [31]:
# 150 den fazla kitap okuyan 3658 kişiye en iyi 500 kitap arasından matrix svd_model e göre tavsiyede bulunacağız.
# Bunu seçmemizin nedeni veri boyutunu küçük tutmak için

df_bookish_user=df_ratings["user_id"].value_counts()
df_bookish_user=df_bookish_user[df_bookish_user>150].index
df_bookish_user

df6=df[df["book_rating"]>4.31][df["book_score"]>df["book_score"].median()]
df6=df6.iloc[:500]
df6

recomended=[]
for user_id in df_bookish_user:
    dfx=df_ratings[df_ratings["user_id"]==user_id]
    for book_id in df6["book_id"]:
        dfy=dfx[dfx["book_id"]==book_id]
        if  len(dfy)>0:
            continue
        est=svd_model.predict(uid=user_id, iid=book_id, verbose=False).est
        recomended.append([user_id,book_id,est])
    #print(len(recomended))
df_recomended=pd.DataFrame(recomended)
df_recomended.columns=["user_id","book_id","est"]
df_recomended    

,user_id,book_id,est
0,12381,1,3.240024
1,12381,2,3.591195
2,12381,18,3.852353
3,12381,21,3.656421
4,12381,23,3.519649
...,...,...,...
1799603,38206,7373,4.682595
1799604,38206,7403,4.373520
1799605,38206,7435,4.571295
1799606,38206,7445,4.165627


In [32]:
#df_recomended.to_csv("df_recomended_matrix.csv")

In [33]:
df_recomended[["user_id"]].nunique()

user_id    3710
dtype: int64

In [34]:
df_recomended[["book_id"]].nunique()

book_id    500
dtype: int64

In [35]:
# Burada en çok kitap okuyan 3658 kişiye en iyi 500 kitap arasından tavsiye veriyoruz.